<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8bcf1a19628cadcb8ed9e108b04a73f6f9e9751b83c5196108f86b058a04c463
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 10:19:50
-------------------
qualified stocks: 90
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.27 L
Current:  1.40 C
-------------------
Today PnL: 26.70 K (0.19%)
Current PnL: -17.96 L (-12.08%)
CY Booked + Current PnL: -8.10 L (-5.45%)
-------------------
Total profit:  2.77 L
Total loss:  -20.73 L
-------------------
Total Booked + Current PnL: 18.88 L (15.52%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.04%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.25 L (56.6%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.18,71.0,H-MC,8.49,143624.0,28688.0,7899.0,-0.29,24.96,5.50,31.83,81.0,3.63,1.07,44.82,X40N,ATH,IT
5,APOLLOHOSP,8285.00,-14.42,57.0,H-LC,10.75,171094.0,14917.0,11172.0,-0.12,9.55,6.53,16.71,22.0,1.34,1.27,28.65,X40N,BTT,HEALTHCARE
80,TTKPRESTIG,770.00,105.76,63.0,M-SC,5.88,88667.0,-12110.0,12201.0,0.68,-12.02,13.76,0.09,245.0,-0.99,0.66,16.49,OX40N,NTT,DURABLES
65,SBIN,863.00,-13.25,46.0,M-LC,4.27,213616.0,12136.0,15081.0,-0.33,6.02,7.06,13.51,60.0,0.80,1.59,19.38,XY25,NTT,BANKS
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.94,190838.0,39925.0,17385.0,0.17,26.46,9.11,37.97,93.0,2.30,1.42,58.45,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,8.03,224428.0,-534.0,168186.0,-0.85,-0.24,74.94,74.53,192.0,-0.00,1.67,31.94,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.50,191355.0,349.0,57349.0,-0.32,0.18,29.97,30.21,157.0,0.01,1.42,21.10,X40N,NTT,FINANCE
37,ICICIGI,2260.25,-21.04,40.0,H-MC,3.72,135790.0,-343.0,31463.0,0.11,-0.25,23.17,22.86,91.0,-0.01,1.01,12.33,X40,ATH,INSURANCE
54,PGHH,17973.08,-31.08,50.0,H-MC,4.80,199515.0,-1305.0,70090.0,0.19,-0.65,35.13,34.25,80.0,-0.02,1.48,4.51,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-2.37,35.0,H-LC,10.19,222704.0,-19220.0,82490.0,1.10,-7.94,37.04,26.15,8.0,-0.23,1.65,5.24,X40,ATH,IT
86,VBL,671.64,-15.98,39.0,H-LC,6.66,302476.0,-13366.0,126042.0,0.01,-4.23,41.67,35.67,5.0,-0.11,2.25,9.04,X40N,ATH,FMCG
46,JIOFIN,387.00,-1.80,41.0,H-LC,13.89,217280.0,-126.0,53625.0,-0.23,-0.06,24.68,24.61,48.0,-0.00,1.61,54.73,XY24,BTT,FINANCE
77,TCS,4998.00,-28.36,42.0,H-LC,14.20,285986.0,-59970.0,183832.0,0.78,-17.33,64.28,35.80,1.0,-0.33,2.12,1.31,X40,BTT,IT
83,UNITDSPR,1644.00,-14.94,42.0,H-LC,8.37,171098.0,-10661.0,45906.0,0.02,-5.87,26.83,19.39,86.0,-0.23,1.27,1.80,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-6.65,48.0,H-LC,1.64,231345.0,-14806.0,94019.0,1.02,-6.02,40.64,32.18,16.0,-0.16,1.72,27.86,X200,ATH,IT
53,NESTLEIND,1377.00,-9.37,56.0,H-LC,2.51,278717.0,13291.0,43508.0,0.29,5.01,15.61,21.40,11.0,0.31,2.07,12.70,XY25,NTT,FMCG
44,ITC,452.00,-38.34,49.0,H-LC,3.24,199732.0,-406.0,21291.0,0.22,-0.20,10.66,10.44,4.0,-0.02,1.48,5.35,X40,NTT,FMCG
20,CIPLA,1795.00,-21.15,45.0,H-LC,5.76,210638.0,6138.0,35282.0,-0.24,3.00,16.75,20.25,10.0,0.17,1.56,12.00,X40N,BTT,PHARMA
86,VBL,671.64,-15.98,39.0,H-LC,6.66,302476.0,-13366.0,126042.0,0.01,-4.23,41.67,35.67,5.0,-0.11,2.25,9.04,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VAIBHAVGBL,521.0,58.68,67.0,H-SC,10.96,105983.0,-46008.0,120132.0,6.96,-30.27,113.35,48.77,125.0,-0.38,0.79,23.36,XR,NTT,JEWELLERY
66,SFL,1287.0,45.97,65.0,M-SC,9.02,192687.0,-69531.0,139351.0,4.28,-26.52,72.32,26.63,229.0,-0.50,1.43,19.80,XY24,NTT,MISC
42,INFY,2275.0,-18.52,51.0,H-LC,10.59,317370.0,4482.0,167191.0,3.98,1.43,52.68,54.87,3.0,0.03,2.36,8.18,X40,BTT,IT
81,UJJIVANSFB,60.0,89.03,54.0,H-SC,12.94,119259.0,-23220.0,42742.0,3.76,-16.30,35.84,13.70,163.0,-0.54,0.89,40.76,OX40N,NTT,BANKS
75,TATAELXSI,9161.0,-20.85,51.0,H-MC,8.36,78883.0,-23769.0,49373.0,3.02,-23.15,62.59,24.94,98.0,-0.48,0.59,19.91,OX40N,NTT,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3743.24,65.0,M-SC,7.78,119782.0,-27586.0,79403.0,-5.17,-18.72,66.29,35.16,236.0,-0.35,0.89,22.35,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-39.64,60.0,L-SC,40.41,90240.0,-23309.0,63358.0,-3.71,-20.53,70.21,35.27,268.0,-0.37,0.67,120.86,XR,NTT,HOTELS
67,SHALBY,327.0,1057.89,70.0,M-SC,2.84,157387.0,-23756.0,69219.0,-3.09,-13.11,43.98,25.10,235.0,-0.34,1.17,26.13,XY24,NTT,HEALTHCARE
13,BATAINDIA,2096.0,-33.15,58.0,M-SC,9.52,99965.0,-28705.0,74004.0,-1.90,-22.31,74.03,35.20,219.0,-0.39,0.74,14.74,X40,NTT,FOOTWEAR
71,STARHEALTH,761.0,8.28,51.0,H-SC,3.75,161009.0,-39207.0,115234.0,-1.78,-19.58,71.57,37.97,144.0,-0.34,1.20,29.28,XY24,NTT,INSURANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TTKPRESTIG,770.0,105.76,63.0,M-SC,5.88,88667.0,-12110.0,12201.0,0.68,-12.02,13.76,0.09,245.0,-0.99,0.66,16.49,OX40N,NTT,DURABLES
81,UJJIVANSFB,60.0,89.03,54.0,H-SC,12.94,119259.0,-23220.0,42742.0,3.76,-16.30,35.84,13.70,163.0,-0.54,0.89,40.76,OX40N,NTT,BANKS
47,KANSAINER,340.0,-67.42,63.0,H-SC,2.60,224001.0,-45666.0,82007.0,2.29,-16.93,36.61,13.47,138.0,-0.56,1.66,13.30,XY24,NTT,PAINTS
40,INDIGOPNTS,1408.0,115.40,51.0,M-SC,3.23,141286.0,-33273.0,33301.0,-0.19,-19.06,23.57,0.02,221.0,-1.00,1.05,22.89,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.81,47.0,H-SC,2.40,114552.0,-29251.0,55993.0,-0.32,-20.34,48.88,18.60,149.0,-0.52,0.85,26.41,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-28.36,42.0,H-LC,14.20,285986.0,-59970.0,183832.0,0.78,-17.33,64.28,35.80,1.0,-0.33,2.12,1.31,X40,BTT,IT
42,INFY,2275.00,-18.52,51.0,H-LC,10.59,317370.0,4482.0,167191.0,3.98,1.43,52.68,54.87,3.0,0.03,2.36,8.18,X40,BTT,IT
44,ITC,452.00,-38.34,49.0,H-LC,3.24,199732.0,-406.0,21291.0,0.22,-0.20,10.66,10.44,4.0,-0.02,1.48,5.35,X40,NTT,FMCG
86,VBL,671.64,-15.98,39.0,H-LC,6.66,302476.0,-13366.0,126042.0,0.01,-4.23,41.67,35.67,5.0,-0.11,2.25,9.04,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,12.77,245424.0,-16195.0,135400.0,0.09,-6.19,55.17,45.57,7.0,-0.12,1.82,4.43,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.94,190838.0,39925.0,17385.0,0.17,26.46,9.11,37.97,93.0,2.30,1.42,58.45,AR,ATH,AUTO
26,DIXON,18940.15,-28.18,71.0,H-MC,8.49,143624.0,28688.0,7899.0,-0.29,24.96,5.50,31.83,81.0,3.63,1.07,44.82,X40N,ATH,IT
27,DMART,5201.00,-6.54,64.0,H-LC,16.58,233436.0,48955.0,21406.0,1.06,26.54,9.17,38.14,38.0,2.29,1.73,39.96,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,8.71,51.0,H-MC,3.35,187750.0,23080.0,44891.0,-0.36,14.02,23.91,41.28,89.0,0.51,1.39,23.12,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.50,191355.0,349.0,57349.0,-0.32,0.18,29.97,30.21,157.0,0.01,1.42,21.10,X40N,NTT,FINANCE
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.94,190838.0,39925.0,17385.0,0.17,26.46,9.11,37.97,93.0,2.30,1.42,58.45,AR,ATH,AUTO
87,VOLTAS,1918.49,0.60,58.0,H-MC,3.49,211155.0,19413.0,76628.0,-0.81,10.12,36.29,50.08,99.0,0.25,1.57,17.61,XY25,ATH,AC
24,DABUR,735.00,-3.12,59.0,H-MC,5.12,211029.0,14001.0,75612.0,-0.93,7.11,35.83,45.49,102.0,0.19,1.57,21.07,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.94,190838.0,39925.0,17385.0,0.17,26.46,9.11,37.97,93.0,2.30,1.42,58.45,AR,ATH,AUTO
52,NATIONALUM,247.44,-44.49,66.0,H-MC,1.73,110118.0,9614.0,21022.0,-1.33,9.57,19.09,30.49,79.0,0.46,0.82,45.07,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.11,102742.0,11244.0,67512.0,0.51,12.29,65.71,86.07,223.0,0.17,0.76,49.20,XR,NTT,DURABLES
39,INDIAMART,4850.92,-51.09,50.0,H-SC,12.13,136724.0,13388.0,120372.0,0.20,10.86,88.04,108.45,119.0,0.11,1.01,35.63,AR,ATH,MISC
89,WIPRO,420.00,-12.43,50.0,M-LC,6.76,154070.0,3125.0,106324.0,2.47,2.07,69.01,72.51,53.0,0.03,1.14,8.22,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6716.67,52.0,L-SC,6.20,74802.0,-19008.0,95627.0,1.26,-20.26,127.84,81.67,269.0,-0.20,0.56,44.86,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.52,40.0,H-SC,14.99,87075.0,-13888.0,108417.0,-0.52,-13.76,124.51,93.63,148.0,-0.13,0.65,29.70,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.41,90240.0,-23309.0,63358.0,-3.71,-20.53,70.21,35.27,268.0,-0.37,0.67,120.86,XR,NTT,HOTELS
51,MASFIN,398.61,-16.89,57.0,H-SC,12.93,94500.0,-3480.0,25080.0,1.50,-3.55,26.54,22.05,152.0,-0.14,0.70,37.36,XR,ATH,FINANCE
85,VALIANTORG,838.00,-333.94,64.0,H-SC,8.87,99554.0,-33732.0,118330.0,2.76,-25.31,118.86,63.47,139.0,-0.29,0.74,66.88,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.41,90240.0,-23309.0,63358.0,-3.71,-20.53,70.21,35.27,268.0,-0.37,0.67,120.86,XR,NTT,HOTELS
8,ATULAUTO,844.00,3743.24,65.0,M-SC,7.78,119782.0,-27586.0,79403.0,-5.17,-18.72,66.29,35.16,236.0,-0.35,0.89,22.35,XY24,NTT,AUTO
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.94,190838.0,39925.0,17385.0,0.17,26.46,9.11,37.97,93.0,2.30,1.42,58.45,AR,ATH,AUTO
63,SAMMAANCAP,326.00,-171.59,64.0,M-SC,4.59,82050.0,-20160.0,113549.0,-0.70,-19.72,138.39,91.37,208.0,-0.18,0.61,33.19,XY25,NTT,FINANCE
27,DMART,5201.00,-6.54,64.0,H-LC,16.58,233436.0,48955.0,21406.0,1.06,26.54,9.17,38.14,38.0,2.29,1.73,39.96,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,42.96
2,50,73.17


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.21
LC    35.52
MC    25.32
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.61
X40N     15.40
X40      15.30
XY25      9.87
AR        9.38
XR        9.15
OX40N     7.42
MH        1.72
X200      1.72
X5K       1.41
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.31
H-SC    23.93
H-MC    22.33
M-SC    13.66
M-LC     7.15
M-MC     2.67
L-SC     1.62
L-LC     1.06
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.56,-0.96,35.61
IT,13.32,-15.08,72.14
FINANCE,8.72,-18.92,61.68
BANKS,7.31,-15.99,65.24
PAINTS,5.98,-14.57,40.21
MISC,5.80,-17.42,84.73
ELECTRICAL,5.59,-9.21,69.40
AUTO,4.83,-4.10,49.39
AC,3.68,2.34,30.73


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2748512.0,22
AR,1214825.0,10
XR,1062062.0,13
X40,903581.0,10
X40N,607356.0,12
OX40N,460835.0,10
XY25,422081.0,7
SR,199167.0,2
X5K,128219.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2801584.0,28
H-LC,1439547.0,16
M-SC,1406386.0,17
H-MC,1196853.0,17
M-LC,416667.0,5
M-MC,318332.0,2
L-SC,244319.0,3
L-MC,59869.0,1
L-LC,41108.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,927082.0,6
M-SC,XY24,818930.0,7
H-SC,AR,659392.0,5
H-LC,X40,605014.0,5
H-SC,XR,501781.0,6
H-MC,XY24,463500.0,4
H-LC,AR,378638.0,2
M-MC,XY24,318332.0,2
H-SC,X40N,249272.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
